In [1]:
import string
import random
import re
from recurrent_network import RNN, RNNLyricsGenerator
from Phyme import Phyme
import torch
from torch.autograd import Variable

In [2]:
def revert(data):
    lines = data.split('\n')
    lines = [x[::-1] for x in lines]
    lines = '\n'.join(lines)
    return lines

In [3]:
def extract_unique_words(data):
    words = set()
    lines = data.split('\n')
    for l in lines:
        line_words = l.split(' ')
        for word in line_words:
            normalized_word = re.sub(r'\W+', '', word).lower()
            words.add(normalized_word)
    return words

In [4]:
def find_rhyme(word):
    rhymes = set()
    candidates = []
    ph = Phyme()    
    rhyme_space = [ph.get_additive_rhymes, ph.get_perfect_rhymes, ph.get_subtractive_rhymes]
    for method in rhyme_space:
        all_rhymes = method(word)
        for syl in all_rhymes:
            for r in all_rhymes[syl]:
                if r in words:
                    rhymes.add(r)
    if len(rhymes) == 0:
        return ''
    return random.choice(list(rhymes))

In [5]:
n_characters = len(string.printable)

with open('DataScraper/all_lyrics.txt', 'r', encoding='utf-8') as f:
    lyrics = f.read()
    reverted_lyrics = revert(lyrics)
len(lyrics)
words = extract_unique_words(lyrics)

In [6]:
n_epochs = 1000
hidden_size = 200
n_layers = 1

In [7]:
forward_decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
backward_decoder = RNN(n_characters, hidden_size, n_characters, n_layers)

In [8]:
forward_generator = RNNLyricsGenerator(forward_decoder, chunk_size=1000)
backward_generator = RNNLyricsGenerator(backward_decoder, chunk_size=1000)

In [9]:
forward_generator.train(lyrics, n_epochs, print_every=500)

 37s (500 50%) 1.7537]
Whit
Fuck the came nogetta litts paying these I dian-Cring lit to to brees
I doncal up the stry
Droom  

 23s (1000 100%) 1.6288]
Whe Guck say 'em notain
Thing you everybody for em
Sh you for they percot you and 'tad, they tat so kn 



In [10]:
backward_generator.train(reverted_lyrics, n_epochs, print_every=100)

 55s (100 10%) 1.9512]
Wh I
hcnof s'ti won I
m'I llail seowt sid eveht tsuac pu wol t'nam ym ....tog dnA
sad od ,tsub I
dlots 

 53s (200 20%) 2.0109]
Wh
ttup ekil ll''nilloc emad eht saw ti tog I ,ay a ,os teg annaa yeht nrukver ahw I
tsal new teg a ,h 

 52s (300 30%) 1.9700]
WhQ p taht kcab snam ym no rof dna yalp reve dna uoy ekil I ti kcil eveno-oocA
?ot ti epmol I tog I su 

 49s (400 40%) 1.6036]
WhU ta saggin I tI
 ruo aggiN
llac ehS
 teg I dluohs ,dic emarc dnA
M esuac annaw ,uoy a ,ti teg flif  

 47s (500 50%) 1.8295]
WhT
drah I
yssarc dellip eht fo lla ot tahw ni saw I neht ekil yarts eht lluoy tib I haey ot ,morf gni 

 48s (600 60%) 2.0379]
WhO
?tup ym lla senog tuo eseht ,no reh evol ,ecnuo tuB
pees ot wonk uoy kcarc os ees annaw uoy ni ew  

 46s (700 70%) 1.7201]
WhO
ti pot taht ekil eksittil ot erehT
 aht kcolG ydob ym meht ym ti tsil m'I
s'taht morf floG
yeh ,uo 

 45s (800 80%) 1.8362]
WhO
tcap ,ag niw emitnals eht tuB
dna ekam skcorb yreve thgil tuB
evael yb tnemod

In [11]:
import pickle

In [12]:
with open('forward_generator.pkl', 'wb') as file:
    pickle.dump(forward_generator, file)

In [13]:
with open('backward_generator.pkl', 'wb') as file:
    pickle.dump(backward_generator, file)

In [14]:
def trim_line(line):
    return line.split('\n')[0]

In [41]:
def remove_half_words(line):
    line = line.strip()
    tokens = line.split(' ')
    first = tokens[0]
    if first not in words:
        res = ' '.join(tokens[1:])
        return res[0].upper() + res[:]
    else:
        return line[0].upper() + line[1:]

In [51]:
def generate_bidirectional_lyrics(forward_generator, backward_generator, n_lines=8):
    lines = []
    while True:
        line = trim_line(forward_generator.generate("Ok", 50, 0.5))
        last_word = line.split(' ')[-1]
        try:
            rhyme = find_rhyme(last_word)
            lines.append(line)
            break
        except:
            continue
    for i in range(1, n_lines):
        while True:
            line = trim_line(revert(backward_generator.generate(revert(rhyme), 60, 0.5)))
            last_word = line.split(' ')[-1]
            try:
                rhyme = find_rhyme(last_word)
                lines.append(line)
                break
            except:
                continue
    return '\n'.join(lines)

In [52]:
print(generate_bidirectional_lyrics(forward_generator, backward_generator, n_lines=16))

Ok now, you got got money at your Pater
t out it bitch in my mothers, and, getties out the just the skater
I want out the waiters
oes all out a greace, in the stries, I need to every in the creators
I give there act, I cashed to, me in my nigga player
I ain't was the take to raider
That's a back out a raider
out and can't best out me make to them dires, I ain't gone, mayer
Fuckin' that me gone fire, thing, I got entertainers
er and there me, why out on the draprects, get me out to my endangers
But they dangers
 and my shit, I'm rapressed to top tell they in my make the pager
I don't give thing in the strangers
I head to this gainers
ing, ever cause roes, have in the mine, man and best have a retainer
that ever and and of that about me out me, I've have that's strangers
